In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
%cd /content/drive/MyDrive/sj/dacon/cancer_transfer_prediction

/content/drive/MyDrive/sj/dacon/cancer_transfer_prediction


In [ ]:
config = {
    'IMG_SIZE': 224,
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
df = pd.read_csv("train.csv")
df.head()

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN,0


In [ ]:
df['암의 장경'] = round(df['암의 장경'].fillna(df['암의 장경'].mean()), 1)
df = df.fillna(0)
df.head()

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,0.0,0.00,0.0,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.00,0.0,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,18.4,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,0.0,0.00,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,0.0,0.0,0.0,1.0,1.0,3.0,0.0,0.00,0.0,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,0.0,8.0,1.0,2.0,1.0,5.44,0.0,0


In [ ]:
df = df.drop(["ID", "mask_path", "수술연월일"], axis=1)
df.head()

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,./train_imgs/BC_01_0001.png,63,1,2,1,19.0,2.0,1.0,2.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,0.0,0.00,0.0,0
1,./train_imgs/BC_01_0002.png,51,1,1,1,22.0,3.0,3.0,3.0,3.0,...,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.00,0.0,1
2,./train_imgs/BC_01_0003.png,37,1,2,1,18.4,2.0,0.0,0.0,0.0,...,7.0,1.0,4.0,7.0,0.0,1.0,0.0,0.00,0.0,0
3,./train_imgs/BC_01_0004.png,54,1,2,1,0.0,3.0,3.0,3.0,3.0,...,0.0,0.0,0.0,1.0,1.0,3.0,0.0,0.00,0.0,0
4,./train_imgs/BC_01_0005.png,57,1,2,1,8.0,2.0,2.0,3.0,2.0,...,8.0,0.0,0.0,8.0,1.0,2.0,1.0,5.44,0.0,0


In [ ]:
numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ["img_path", "N_category"]
for col in df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        df[col] = (df[col] - df[col].mean()) / df[col].std()

df.head()

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,./train_imgs/BC_01_0001.png,0.828278,1,2,1,0.053617,2.0,1.0,2.0,2.0,...,0.918130,1.0,0.741666,-0.098976,0.0,1.0,0.0,-0.133450,0.0,0
1,./train_imgs/BC_01_0002.png,-0.238060,1,1,1,0.311927,3.0,3.0,3.0,3.0,...,-1.475170,0.0,-0.911993,3.092933,0.0,0.0,0.0,-0.133450,0.0,1
2,./train_imgs/BC_01_0003.png,-1.482121,1,2,1,0.001955,2.0,0.0,0.0,0.0,...,0.618967,1.0,0.190446,-0.374140,0.0,1.0,0.0,-0.133450,0.0,0
3,./train_imgs/BC_01_0004.png,0.028525,1,2,1,-1.582349,3.0,3.0,3.0,3.0,...,-1.475170,0.0,-0.911993,-0.704338,1.0,3.0,0.0,-0.133450,0.0,0
4,./train_imgs/BC_01_0005.png,0.295109,1,2,1,-0.893522,2.0,2.0,3.0,2.0,...,0.918130,0.0,-0.911993,-0.319107,1.0,2.0,1.0,1.552578,0.0,0


In [ ]:
df.shape

(1000, 25)

In [ ]:
df = df.sample(frac=1, random_state=config['SEED']).reset_index(drop=True)
df_train, df_valid = df.iloc[:int(len(df) * 0.8)], df.iloc[int(len(df) * 0.8):]
df_train.shape, df_valid.shape

((800, 25), (200, 25))

In [ ]:
df_train["N_category"].value_counts().sort_index()

0    397
1    403
Name: N_category, dtype: int64

In [ ]:
# generate image, tabluar, label
def generator_train():
    train = df_train.sample(frac=1)
    for item in train.values:
        yield (item[0], item[1:-1], item[-1])

def generator_valid():
    for item in df_valid.values:
        yield (item[0], item[1:-1], item[-1])
    
dataset_train = tf.data.Dataset.from_generator(
    generator_train,
    (tf.string, tf.float32, tf.int32),
    ((), (23,), ())
    )

dataset_valid = tf.data.Dataset.from_generator(
    generator_valid,
    (tf.string, tf.float32, tf.int32),
    ((), (23,), ())
    )

def augmentation(image):
    # add padding
    h, w = image.shape[0], image.shape[1]
    size = h if h >= w else w
    dst = tf.image.pad_to_bounding_box(image, int((size-h)/2), int((size-w)/2), size, size)
    dst = tf.image.resize(dst, size=(config['IMG_SIZE'], config['IMG_SIZE']))

    # image augmentation
    dst = tf.image.random_crop(dst, size=(int(config['IMG_SIZE'] * 0.9), int(config['IMG_SIZE'] * 0.9), 3))
    dst = tf.image.resize(dst, size=(config['IMG_SIZE'], config['IMG_SIZE']))
    dst = tf.image.random_flip_left_right(dst)
    dst = tf.cast(dst, tf.float32)
    dst /= 255
    
    return dst

def preprocessing(path, category, label):
    bin = tf.io.read_file(path)
    image = tf.io.decode_png(bin, channels=3)
    image= tf.py_function(augmentation, [image], [tf.float32])
    return (image, category), label

dt = dataset_train.map(preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dt = dt.batch(config['BATCH_SIZE']).prefetch(3)
dv = dataset_valid.map(preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dv = dv.batch(config['BATCH_SIZE']).prefetch(3)

In [ ]:
for (image, clinical), label in dt.take(1):
    print(image[0].shape)
    print(clinical[0])
    print(label)
    plt.imshow(image[0][0])
    plt.show()

In [ ]:
mobilenet = tf.keras.applications.MobileNetV3Large(include_top=False, input_shape=[config['IMG_SIZE'], config['IMG_SIZE'], 3], weights='imagenet')

input1 = tf.keras.layers.Input(shape=(config['IMG_SIZE'], config['IMG_SIZE'], 3), dtype=tf.float32)

x = tf.keras.applications.mobilenet_v3.preprocess_input(input1)
x = tf.squeeze(x)
x = mobilenet(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)

input2 = tf.keras.layers.Input(shape=(23,), dtype=tf.float32)
x = tf.keras.layers.concatenate([x, input2])
output = tf.keras.layers.Dense(2, activation='softmax')(x)

#model
model = tf.keras.models.Model(inputs=[input1, input2], outputs=[output])

12683000/12683000 [==============================] - 2s 0us/step


In [ ]:
mc = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/sj/dacon/cancer_transfer_prediction/model_best.h5", 
                                        monitor='val_loss', mode='min', verbose=1, save_best_only=True)

mobile.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=['val_acc'])
model.fit((dt), epochs=10, validation_data=(dv), callbacks=[mc])

Epoch 1/10
     25/Unknown - 52s 2s/step - loss: 0.5087 - sparse_categorical_accuracy: 0.7437
Epoch 1: val_loss improved from inf to 0.90315, saving model to /content/drive/MyDrive/sj/dacon/cancer_transfer_prediction/model_best.h5
25/25 [==============================] - 66s 2s/step - loss: 0.5087 - sparse_categorical_accuracy: 0.7437 - val_loss: 0.9032 - val_sparse_categorical_accuracy: 0.4450
Epoch 2/10
25/25 [==============================] - ETA: 0s - loss: 0.4411 - sparse_categorical_accuracy: 0.7975
Epoch 2: val_loss improved from 0.90315 to 0.89847, saving model to /content/drive/MyDrive/sj/dacon/cancer_transfer_prediction/model_best.h5
25/25 [==============================] - 59s 2s/step - loss: 0.4411 - sparse_categorical_accuracy: 0.7975 - val_loss: 0.8985 - val_sparse_categorical_accuracy: 0.4450
Epoch 3/10
25/25 [==============================] - ETA: 0s - loss: 0.3750 - sparse_categorical_accuracy: 0.8112
Epoch 3: val_loss improved from 0.89847 to 0.79714, saving model to 